In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
import pandas as pd
import plotly.express as px
import base64
from dash.dependencies import Input, Output, State
from pymongo import MongoClient
from bson.objectid import ObjectId
import os


#########################
# Dashboard Layout / View
#########################


image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

df = pd.read_csv('aac_shelter_outcomes.csv')

app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'width': '200px', 'height': '200px'}
    ),
    html.Center(html.B(html.H2('Austin Animal Operation Dashboard'))),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Button(id='button-one', n_clicks=0, children='Water Rescue'),
            html.Button(id='button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
            html.Button(id='button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
            html.Button(id='button-four', n_clicks=0, children='reset')
        ]
    ),
    html.Hr(),
    html.Div(
        className='row',
        children=[
            html.Div(
                id='datatable-id-div',
                className='col s12',
                style={'width': '100%'},
                children=[
                    dash_table.DataTable(
                        id='datatable-id',
                        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                        data=df.to_dict('records'),
                        page_size=100,
                        style_table={'height': '400px', 'overflowY': 'auto', 'overflowX': 'auto'},
                        style_header={
                            'backgroundColor': 'rgb(240, 230, 230)',
                            'fontWeight': 'bold'
                        },
                        style_data={
                            'whiteSpace': 'normal',
                            'height': 'auto'
                        },
                        # tooltips
                        tooltip={i: i for i in df.columns},
                        tooltip_delay=0,
                        tooltip_duration=None,
                        editable=False,
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable=False,
                        row_selectable=False,
                        row_deletable=False,
                        selected_columns=[],
                        selected_rows=[],
                        page_action="native",
                        page_current=0,
                    ),
                ]
            ),
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={'width': '50%', 'display': 'inline-block'}
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '50%', 'height': '500px', 'display': 'inline-block'}
            )
        ]
    ),
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('button-one', 'n_clicks'),
     Output('button-two', 'n_clicks'),
     Output('button-three', 'n_clicks')],
    [Input('button-one', 'n_clicks'),
     Input('button-two', 'n_clicks'),
     Input('button-three', 'n_clicks'),
     Input('button-four', 'n_clicks')],
    [State('button-one', 'n_clicks_timestamp'),
     State('button-two', 'n_clicks_timestamp'),
     State('button-three', 'n_clicks_timestamp'),
     State('button-four', 'n_clicks_timestamp')]
)
def update_dashboard(button1, button2, button3, button4, timestamp1, timestamp2, timestamp3, timestamp4):
    # Get the most recent timestamp among all buttons
    latest_timestamp = max(timestamp1 or 0, timestamp2 or 0, timestamp3 or 0, timestamp4 or 0)

    # Determine which buttons were clicked
    clicked_button = None
    if latest_timestamp == timestamp1:
        clicked_button = 'button-one'
    elif latest_timestamp == timestamp2:
        clicked_button = 'button-two'
    elif latest_timestamp == timestamp3:
        clicked_button = 'button-three'

    # Reset all button states to zero when the "reset" button is clicked
    if clicked_button == 'button-four':
        return df.to_dict('records'), 0, 0, 0

    # Rest of your filtering code remains the same
    if clicked_button == 'button-one':
        # Apply the first filter condition
        df_filtered = df[df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']) &
                         (df['sex_upon_outcome'] == 'Intact Female') &
                         (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif clicked_button == 'button-two':
        # Apply the second filter condition
        df_filtered = df[df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']) &
                         (df['sex_upon_outcome'] == 'Intact Male') &
                         (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif clicked_button == 'button-three':
        # Apply the third filter condition
        df_filtered = df[df['breed'].isin(['Doberman Pinscher', 'German Sheperd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']) &
                         (df['sex_upon_outcome'] == 'Intact Male') &
                         (df['age_upon_outcome_in_weeks'].between(20, 300))]
    else:
        # If no buttons are clicked, return the original DataFrame
        df_filtered = df

    return df_filtered.to_dict('records'), 0, 0, 0


    # Rest of your filtering code remains the same
    if clicked_button == 'button-one':
        # Apply the first filter condition
        df_filtered = df[df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']) &
                         (df['sex_upon_outcome'] == 'Intact Female') &
                         (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif clicked_button == 'button-two':
        # Apply the second filter condition
        df_filtered = df[df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']) &
                         (df['sex_upon_outcome'] == 'Intact Male') &
                         (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif clicked_button == 'button-three':
        # Apply the third filter condition
        df_filtered = df[df['breed'].isin(['Doberman Pinscher', 'German Sheperd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']) &
                         (df['sex_upon_outcome'] == 'Intact Male') &
                         (df['age_upon_outcome_in_weeks'].between(20, 300))]
    else:
        # If no buttons are clicked, return the original DataFrame
        df_filtered = df

    return df_filtered.to_dict('records'), 0, 0, 0


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    df_filtered = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(df_filtered, names='breed', title='Preferred Animals')
        )
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.read_csv('aac_shelter_outcomes.csv')

# Austin TX is at [30.75,-97.48]
    return [
    dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
       dl.TileLayer(id="base-layer-id"),
       dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
          dl.Tooltip(dff.iloc[0,4]),
          dl.Popup([
             html.H1("Animal Name"),
            html.P(dff.loc[0,'name'])
         ])
      ])
   ])
]

if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:19189/
